<a href="https://colab.research.google.com/github/villasen/colab_notebooks/blob/master/dscnn_speech_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -r work

In [0]:
!rm -r ML-KWS-for-MCU

In [0]:
!git clone https://github.com/villasen/ML-KWS-for-MCU.git
  

Train with default word set

In [0]:
!python ML-KWS-for-MCU/train.py --model_architecture ds_cnn --model_size_info 5 64 10 4 2 2 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 --dct_coefficient_count 10 \
--window_size_ms 40 --window_stride_ms 20 --learning_rate 0.0005,0.0001,0.00002 --how_many_training_steps 2000,1000,1000 --summaries_dir work/DS_CNN/DS_CNN1/retrain_logs \
--train_dir work/DS_CNN/DS_CNN1/training 


Train with alternate word set

In [0]:
!python ML-KWS-for-MCU/train.py --model_architecture ds_cnn --model_size_info 5 64 10 4 2 2 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 --dct_coefficient_count 10 \
--window_size_ms 40 --window_stride_ms 20 --learning_rate 0.0005,0.0001,0.00002 --how_many_training_steps 5000,3000,2000 --summaries_dir work/DS_CNN/DS_CNN1/retrain_logs \
--train_dir work/DS_CNN/DS_CNN1/training --wanted_words=on,bed,cat,dog,down,eight,five,follow,off,four 


Test model

In [0]:
!python ML-KWS-for-MCU/test.py --data_dir=/tmp/speech_dataset/ --model_architecture ds_cnn --model_size_info 5 64 10 4 2 2 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 \
-dct_coefficient_count 10 --window_size_ms 40 --window_stride_ms 20 --checkpoint work/DS_CNN/DS_CNN1/training/best/ds_cnn_9019.ckpt-3600


Freeze graph to protobuf file

In [0]:
!python ML-KWS-for-MCU/freeze.py --model_architecture ds_cnn --model_size_info 5 64 10 4 2 2 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 --dct_coefficient_count 10 \
--window_size_ms 40 --window_stride_ms 20 --learning_rate 0.0005,0.0001,0.00002 --how_many_training_steps 10000,10000,10000 --summaries_dir work/DS_CNN/DS_CNN1/retrain_logs \
--train_dir work/DS_CNN/DS_CNN1/training --wanted_words=on,bed,cat,dog,down,eight,five,follow,off,four --checkpoint work/DNN/DNN1/training/best/    --output_file dscnn_dataset.pb


Fold Batchnorm (needed for quantization of ds-cnn)

In [0]:
!python ML-KWS-for-MCU/fold_batchnorm.py --model_architecture ds_cnn --model_size_info 5 64 10 4 2 2 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 \
--dct_coefficient_count 10 --window_size_ms 40  --window_stride_ms 20 --checkpoint work/DS_CNN/DS_CNN1/training/best/ds_cnn_9019.ckpt-3600



Quantize model to weights.h file

In [0]:
!python ML-KWS-for-MCU/quant_test.py --data_dir=/tmp/speech_dataset/ --model_architecture ds_cnn --model_size_info 5 64 10 4 2 2 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 64 3 3 1 1 \
--dct_coefficient_count 10 --window_size_ms 40  --window_stride_ms 20 --checkpoint work/DS_CNN/DS_CNN1/training/best/ds_cnn_9019.ckpt-3600_bnfused --act_max 32 0 0 0 0 0 0 0 0 0 0 0

Download weights.h

In [0]:
from google.colab import files

files.download('weights.h')

Download model as tar file

In [0]:
!tar -cvf dscnn__defaultDataset_4000.tar /content/work/
!ls -l

from google.colab import files

files.download('dscnn__defaultDataset_4000.tar')

In [0]:
!ls -l /content/work/DS_CNN/DS_CNN1/training/
!cp /content/work/DS_CNN/DS_CNN1/training/ds_cnn_labels.txt /content/temp/
!ls /content/temp/

In [0]:
!cp /content/work/DS_CNN/DS_CNN1/training/ds_cnn.pbtxt /content/temp/
!ls /content/temp/